This notebook assumes that we've already run **VF vs Seneca - Part 1**. If you haven't already created `token_table.csv`, you can use the cell below to execute the whole notebook and create the file.

In [ ]:
%%capture
%run "03 - VF vs Seneca - Part 1.ipynb"

## Getting started

### `import` statements

In [ ]:
# tools for tabular data
import pandas as pd

# plotting
from matplotlib import pyplot as plt

### Load the token table from Part 1

Here we use Pandas to read in the CSV file that we exported at the end of the previous notebook.

In [ ]:
token_table = pd.read_csv('token_table.csv')
display(token_table)

It looks more or less the same, except that where the previous table showed missing data as Python's special value `None`, we now see `NaN` ("not a number"), a value used by Pandas for the same purpose.

## Summarizing the data

Let's look at a couple of simple summaries of the token table. Can we count the number of tokens used by each text? By each author?

We'll use the Pandas [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) method combined with [`agg()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html) — i.e. "aggregate".

The first method, **groupby**, takes as its argument a column name or list of column names to group by. It produces a collection of named **groups**, subsets of the original table.

The second method, **aggregate**, combines (aggregates) records to create new values. Its arguments are a little complicated. They appear as key-value pairs:
- The key (on the left side of the `=`) is the name of a new column
- The value is a **tuple**, a pair of values enclosed in parentheses
    - the first value is the name of the column whose values we want to combine
    - the second value is the name of an aggregation function
    
In the example below, I'm grouping by **author**. Then I'm aggregating to count the number of tokens in each group. My new column name is **n_tokens**, the column I'm aggregating is **token**, and the aggregation function is `count`.

In [ ]:
token_table.groupby('author').agg(n_tokens=('token', 'count'))

Now let's break the data down in a little more detail. This time, we'll group by two columns, **author** and **title**. The column names appear as a list here. My aggregation step is the same.

In [ ]:
token_table.groupby(['author', 'title']).agg(n_tokens=('token', 'count'))

### From tables to graphs

Python allows us to draw some relatively sophisticated plots right here in our notebook, with fine-grained control over things like data series, plot type, titles, legend, etc.

But we can also get some quick-and-dirty plots quite easily just using our Pandas data frame. Here, I'm going to append a plotting method, `plot.barh()`, to the code we just used above to produce a quick comparison of the text lengths in tokens.

In [ ]:
token_table.groupby(['author', 'title']).agg(n_tokens=('token', 'count')).plot.barh()

. . . But before we worry too much about customizing the output, let's first continue with exploring the data in tabular form.

## Counting feature frequencies

At the heart of much stylometry is the counting occurrences of **features**, aspects of the text in which we're interested. These are often words, lemmata, part of speech tags, or morphological features. They can also be sub-word features like characters, phonemes, metrical quantities, etc., or larger features such as n-grams (groups of words).

### Word distribution

One easy place to start is with the most frequent words. [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law) tells us that the most common words occurr exponentially more frequently than less-common words.

The top words tend to be so-called **function words**, words that are largely independent of the subject being written about and more related to the way the language is put together. These words are of particular interest to authorship attribution, which aims to recognize authorial style regardless of the subject.

On the other end, there tends to be a very long tail of **hapax legomena**, words that are only used once. Drawing conclusions from the use of very rare words is tempting, but difficult to justify statistically.

Somewhere in the middle are **content words** words that have some (more or less?) connection to the subject at hand. These words are of interest to information retrieval—for example, finding documents in an archive that are all on the same subject, despite being by different authors or written in different styles.

### Counting words

Let's start by calculating how many time each distinct word form occurs. Pandas data frames have a  `value_counts()` method, which counts how many times each value occurs in a given column(s).

In [ ]:
token_count = token_table.value_counts('token')
display(token_count[:25])

For our purposes here, I'd be happy to drop the punctuation marks. We can apply a filter to the original `token_table` to produce a new table that leaves out any rows where column **upos** has the value `PUNCT`:

In [ ]:
no_punct = token_table.loc[token_table.upos != 'PUNCT']

Now let's try calculating the top tokens again:

In [ ]:
token_count = no_punct.value_counts('token')
display(token_count[:25])

### Does this look right?

#### MFWs

The most frequent words (MFW) are indeed function words, as expected. Words like *que*, *et*, *in*, *est*, *non*, can be expected to be among the top regardless of whether we're reading Cicero, Vergil, or Augustine. But as it turns out, the specific relative frequencies of these words can be a strong marker of authorial style...

#### Zipf's Law

If Zipf's Law holds true, then frequency decreases exponentially with rank. If we plot these counts on a log-log scale, with the most frequent at the left, then we should see a more or less straight line angling down from the upper-left to the lower-right.

In [ ]:
# create a new plot
fig, ax = plt.subplots()

# plot counts vs rank on a log-log plot
ax.loglog(range(len(token_count)), token_count)

# set some labels
ax.set_ylabel('frequency')
ax.set_xlabel('rank')
ax.set_title('Token Frequency vs. Rank in VF and Seneca')

# show
plt.show()

### From inflected words to lemmata

Most of the time, we aren't interested in counting, e.g., *est* and *sunt*, or *me* and *mihi*, independently. By consolidating all inflected forms of the same dictionary headword, we can reduce the number of features and increase their counts.

We'll use exactly the same method, but working from the **lemma** column instead of **token**.

In [ ]:
lemma_count = no_punct.value_counts('lemma')
display(lemma_count[:25])

### 🤔 Does this make sense?

The top two tokens, *que* and *et*, appear more or less unchanged here. But the next two lemmata, *qui* and *sum*, represent productive paradigms with lots of different forms. Members show up in the top tokens, *est* at 678 for example, or *quae* at 269 and *qui* at 253. But when their individual forms are consolidated, these lemmata rise in the list.

### Predominance of function words

The top of the list is still dominated by function words. Just looking at this lexicon, you wouldn't guess that our corpus was made up of Imperial tragedy and epic, would you? The only noun I see in the top 25 is *manus*, and the only verbs are *sum*, *do*, and *video*, all pretty generic.

### The long tail

Even after lemmatization, we still have an enormous feature set here, and a large proportion of them still occur only once.

In [ ]:
# how many lemmata?
print(len(lemma_count), 'lemmata')

In [ ]:
# how many occur only once?
n_hapax = (lemma_count==1).sum()
print(n_hapax, 'hapax legomena')

**Wow:** half of the lemmata in our table only occur once time! These are unlikely to be useful in comparisons between texts.

## More features

### Part of speech tags

Let's carry on with our exploration of feature counts and examine the **upos** column in our big token table.

In [ ]:
pos_count = no_punct.value_counts('upos')
display(pos_count[:25])

### Morphological features

Similarly, we can count features derived from SpaCy's `morph` attribute. Things like mood, tense, and case can add back into the feature set information that was lost through lemmatization, but again in a consolidated way that keeps number of features low and counts high.

In [ ]:
# mood
display(no_punct.value_counts('mood'))

# tense
display(no_punct.value_counts('tense'))

# case
display(no_punct.value_counts('case'))

## Comparison between authors

Let's see whether we can discover any quantifiable differences between Valerius Flaccus and Seneca on the basis of authorial style. Let's start by comparing the use of different parts of speech.

We can use the Pandas `crosstab()` method to tabulate how many times each value in a particular column, say **upos** intersects with each value in another column, say **author**, to tabulate how many times each part of speech occurs in each author.

In [ ]:
pd.crosstab(no_punct.author, no_punct.upos)

This tells us that Seneca uses more adjectives in his tragedies than Valerius Flaccus does in the *Argonautica*, but it doesn't give us a sense of how important these differences are, given the different lengths of the texts and the variation within each author.

### Sampling

Let's represent each author not as a single bag of words, but as a corpus of books, or, in more general terms, samples. We'll throw the **title** column into `crosstab()` along with author to cut the table more finely. This gives us a better sense of how much each author varies.

In [ ]:
pd.crosstab([no_punct.author, no_punct.title], no_punct.upos)

### Normalization

We still need to adjust for the fact that some books are longer than others. To make these features comparable, we should represent them as **frequencies** rather than as raw **counts**. This process is often called **normalization**. Pandas `crosstab()` includes an option to normalize across rows in the output table, so that each row is a series of fractions that all add to 1.

In [ ]:
pos_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct.upos, normalize='index')
display(pos_freq)

Now we can see that Seneca does indeed tend to use more adjectives than Valerius Flaccus, although their practice sometimes overlaps. One way we can represent this difference in distributions is with a histogram.

In [ ]:
# create empty list for the labels and data values
labels = []
values = []

# extract labels, data from grouped table
for label, group in pos_freq.groupby('author'):
    labels.append(label)
    values.append(group['ADJ'])

# create a new plot
fig, ax = plt.subplots()

# plot the values
ax.violinplot(values)

# add labels
ax.set_xticks(range(1, len(labels)+1), labels=labels)
ax.set_ylabel('frequency')
ax.set_title('adjective use by Seneca and VF')

# show
plt.show()



Here's a more generic version of the same code, with the column selection placed at the very top. Try changing this value to explore differences between the two authors.

In [ ]:
# feature of interest
feat = 'NOUN'

# create empty list for the labels and data values
labels = []
values = []

# extract labels, data from grouped table
for label, group in pos_freq.groupby('author'):
    labels.append(label)
    values.append(group[feat])

# create a new plot
fig, ax = plt.subplots()

# plot the values
ax.violinplot(values)

# add labels
ax.set_xticks(range(1, len(labels)+1), labels=labels)
ax.set_ylabel('frequency')
ax.set_title(f'{feat} use by Seneca and VF')

# show
plt.show()
